In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
cards_csv = pd.read_csv("/kaggle/input/cards-image-datasetclassification/cards.csv")
cards_csv.head()

,class index,filepaths,labels,card type,data set
0,0,train/ace of clubs/001.jpg,ace of clubs,ace,train
1,0,train/ace of clubs/002.jpg,ace of clubs,ace,train
2,0,train/ace of clubs/003.jpg,ace of clubs,ace,train
3,0,train/ace of clubs/004.jpg,ace of clubs,ace,train
4,0,train/ace of clubs/005.jpg,ace of clubs,ace,train


In [3]:
# Transformações de pré-processamento se necessário.
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [4]:
# Conjuntos de dados para treinamento e teste:
# Transformar rgb in greyscale !!!!!!!
train_dataset = datasets.ImageFolder('/kaggle/input/cards-image-datasetclassification/train', transform=transform)
test_dataset = datasets.ImageFolder('/kaggle/input/cards-image-datasetclassification/test', transform=transform)

In [5]:
# Crie carregadores de dados para treinamento e teste:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


for X, y in test_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([32, 1, 128, 128])
Shape of y: torch.Size([32]) torch.int64


In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Definindo o modelo

In [7]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(128*128, 512),  # Dúvida aqui (Valores passados!) !!!
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 53)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=16384, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=53, bias=True)
  )
)


# Otimizando os parâmetros do modelo

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer):   # Dúvida nessa etapa !!!
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test(dataloader, model, loss_fn):   
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n------------------------------>")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)
print("Done!")

Epoch 1
------------------------------>
loss: 3.975934  [   32/ 7624]
loss: 3.966501  [ 3232/ 7624]
loss: 3.969722  [ 6432/ 7624]
Test Error: 
 Accuracy: 3.8%, Avg loss: 3.965639 

Epoch 2
------------------------------>
loss: 3.982191  [   32/ 7624]
loss: 3.975721  [ 3232/ 7624]
loss: 3.966640  [ 6432/ 7624]
Test Error: 
 Accuracy: 3.0%, Avg loss: 3.961413 

Epoch 3
------------------------------>
loss: 3.974455  [   32/ 7624]
loss: 3.985400  [ 3232/ 7624]
loss: 3.956096  [ 6432/ 7624]
Test Error: 
 Accuracy: 4.9%, Avg loss: 3.956224 

Epoch 4
------------------------------>
loss: 3.954335  [   32/ 7624]
loss: 3.968601  [ 3232/ 7624]
loss: 3.952565  [ 6432/ 7624]
Test Error: 
 Accuracy: 5.7%, Avg loss: 3.953474 

Epoch 5
------------------------------>
loss: 3.948606  [   32/ 7624]
loss: 3.932954  [ 3232/ 7624]
loss: 3.949716  [ 6432/ 7624]
Test Error: 
 Accuracy: 4.5%, Avg loss: 3.947773 

Done!


In [12]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# Loading Model

In [13]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [14]:
classes = cards_csv['labels'].unique()

model.eval()
x, y = test_dataset[0]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "ace of spades", Actual: "ace of clubs"
